In [19]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.pre_tokenizers import WhitespaceSplit
import tokenizers.decoders as decoders
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.decoer = decoders.BPEDecoder()

# tokenizer = Tokenizer(Unigram())
# tokenizer = Tokenizer(WordPiece(unk_token="<unk>",))
# tokenizer.decoder = decoders.WordPiece()

tokenizer.pre_tokenizer = WhitespaceSplit()

In [20]:
vocab_size = 5000
special_tokens = ["<s>", "<pad>", "</s>", "<unk>"]
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=vocab_size, special_tokens=special_tokens, show_progress=True)
# from tokenizers.trainers import UnigramTrainer
# trainer = UnigramTrainer(special_tokens=special_tokens, show_progress=True)
# from tokenizers.trainers import WordPieceTrainer
# trainer = WordPieceTrainer(special_tokens=special_tokens, show_progress=True)


In [21]:
def gen():
    import json
    data = json.load(open("all.json", "r"))
    for d in data:
        yield d["plain_text"]
        yield d["tokenized_kana_text"]

In [22]:
tokenizer.train_from_iterator(gen(), trainer=trainer)

In [23]:
from tokenizers.processors import TemplateProcessing

tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> </s> <s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ],)

In [24]:
tokenizer.save("tokenizer.json")

In [25]:
text1 = "エ+F ー+F | コ ノ | ダ イ メ ー カ ラ ー | ワ タ シ ガ | ナ ニ オ | ヤ リ タ イ カ ッ テ ユ ー ノ ガ | チョ ッ ト | ワ カ リ ズ ラ イ ト | オ モ ウ ン デ ス ケ レ ド モ <sp> エ+F ー+F | コ ノ | ケ ン キュ ー オ ト ー シ テ | ワ タ シ ガ ー | ヤ リ タ カ ッ タ | コ ト ト ユ ー ノ ワ"
text2 = "エ+F ー+F ト+F | オ ン セ ー ト | モ ジ ー ト | ハ ツ ワ ウ ン ド ー ノ+D ト ノ | カ ン ケ ー オ | ア キ ラ カ ニ | ス ル | コ ト デ ス"
text3 = "それではこれから発表を始めたいと思います"
encoded = tokenizer.encode(text3)

In [30]:
tokenizer.decode(encoded.ids, skip_special_tokens=True)

'それ では これから 発表 を 始め たい と思います'

In [31]:
encoded.ids

[0, 3316, 3359, 4212, 3866, 125, 4982, 3428, 3420, 2]